In [4]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re

data_folder = 'data_last_congress/'

In [2]:
with open('api_key.txt') as f:
    API_KEY = f.readline().rstrip()
f.close()
HEADERS = {"X-Api-Key": API_KEY}

In [3]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder(data_folder)
# create_folder(data_folder + 'commitees')
# create_folder(data_folder + 'senate_members')
# create_folder(data_folder + 'votes')
# create_folder(data_folder + 'lobby')

# Get information from the API

###  Get information about Bills
We need the chamber member that sponsored it and the subject. It is possible that some members will not have any sponsorship, in which case they should be ommitted from the dataviz?

In the future, instead of looking at sponsorship (1 member/bill), we could look at cosponsorship (several members/bill). In order to do that we need to do a query for the cosponsors of each specific bill.

In [4]:
file_name_bills = 'recent_bills.csv'

req_str = "https://api.propublica.org/congress/v1/{congress}/{chamber}/bills/{type_}.json"
# Parameters to configure the request
congresses = [115]
chambers = ['house', 'senate']
type_s = ['passed', 'updated', 'active',
          'introduced', 'ennacted', 'vetoed']

# The first element contains the json field with the results
# The remaining elements contain information added as columns to the df
bill_list_fields = ['bills', 'congress', 'chamber', 'type_']
# Relevant fields for each bill
bill_fields = ['bill_id', 'title',
    'sponsor_id', 'sponsor_name', 'sponsor_state', 'sponsor_party', 
    'primary_subject',
    'cosponsors',
    'committees', 'committee_codes', 'bill_slug', 'bill_type']

header_written = False

for congress in congresses:
    for chamber in chambers:
        for type_ in type_s:
            res = requests.get(
                req_str.format(
                    congress=congress,
                    chamber=chamber,
                    type_=type_),
                headers = HEADERS)
            try:
                res_df = pd.io.json.json_normalize(
                res.json()['results'][0][bill_list_fields[0]])
            
                clean_df = res_df[bill_fields]

                # Add bill_list information
                if len(bill_list_fields):
                    clean_df = clean_df.assign(
                        **{f:eval(f) for f in bill_list_fields[1:]})

                # Save to file
                if not header_written:
                    # Write the header (only once)
                    f = open(data_folder + file_name_bills, "a+")
                    f.write(clean_df.head(0).to_csv(index=False))
                    header_written = True
                f.write(clean_df.to_csv(header=False, index=False))
            except:
                print(congress, chamber, type_)
                print(res)
                print(res.json())
f.close()

115 house ennacted
<Response [200]>
{'status': '500', 'error': 'Internal Server Error'}
115 house vetoed
<Response [200]>
{'status': 'OK', 'copyright': 'Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.', 'results': [{'congress': 115, 'chamber': 'House', 'num_results': 0, 'offset': 0, 'bills': []}]}
115 senate ennacted
<Response [200]>
{'status': '500', 'error': 'Internal Server Error'}
115 senate vetoed
<Response [200]>
{'status': 'OK', 'copyright': 'Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.', 'results': [{'congress': 115, 'chamber': 'Senate', 'num_results': 0, 'offset': 0, 'bills': []}]}


###  Get lists of members, per chamber
This is in order to include more details about the person

The results include all members who have served in that congress and chamber, including members who are no longer in office.

###  TODO: Get information about the Congress

In [6]:
# president name
# start and end dates

# Process data that we need

In [6]:
file_name_bills = 'recent_bills.csv'
bills_df = pd.read_csv(data_folder + file_name_bills)

In [7]:
bills_df.head(5)`````````````````````````````

,bill_id,title,sponsor_id,sponsor_name,sponsor_state,sponsor_party,primary_subject,cosponsors,committees,committee_codes,bill_slug,bill_type,congress,chamber,type_
0,s3554-115,A bill to extend the effective date for the su...,K000393,John Kennedy,LA,R,Commerce,2,Senate Small Business and Entrepreneurship Com...,['SSSB'],s3554,s,115,house,passed
1,hr6784-115,To provide for removal of the gray wolf in the...,D000614,Sean Duffy,WI,R,Animals,3,House Natural Resources Committee,['HSII'],hr6784,hr,115,house,passed
2,hr6116-115,To designate the facility of the United States...,M001194,John Moolenaar,MI,R,Government Operations and Politics,12,Senate Homeland Security and Governmental Affa...,"['SSGA', 'HSGO']",hr6116,hr,115,house,passed
3,hr5935-115,To designate the facility of the United States...,D000619,Rodney Davis,IL,R,Government Operations and Politics,17,Senate Homeland Security and Governmental Affa...,"['SSGA', 'HSGO']",hr5935,hr,115,house,passed
4,hr5868-115,To designate the facility of the United States...,G000563,Bob Gibbs,OH,R,Government Operations and Politics,14,Senate Homeland Security and Governmental Affa...,"['SSGA', 'HSGO']",hr5868,hr,115,house,passed


### Subjects

In [8]:
bills_df.groupby(by=["primary_subject", "sponsor_state", "sponsor_name"])